In [2]:
import json
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

from tqdm import tqdm

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Debt forecast using World Bank data
- This notebook is an attempt generating international debt forecasts for various low & middle income countries using data from World Bank API.
- Data was extracted using a previous notebook in this repository
- Dataset reference: https://wbdata.readthedocs.io/en/latest/#
- Three different time series forecasting models where used: ARMA, ARIMA and SARIMA
- Plot all forecasts to file
- Code is a bit redundant and there are multiple ways to improve but does the job

In [4]:
# load world bank data
wb_df = pd.read_csv("debt_data.csv",index_col="date", infer_datetime_format=True, parse_dates=True)

In [5]:
# resample to enforce index frequency
# needed for later steps
# reference to frequency strings: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects
wb_df = wb_df.resample("AS").mean()
# fill NaNs with bfills
wb_df = wb_df.fillna(method="bfill")
# drop cols that only contain NaNs
wb_df = wb_df.dropna(how='all', axis=1)
# drop rows that only contain NaNs
wb_df = wb_df.dropna(how='all', axis=0)
# convert column types from np.float64 to np.int64
wb_df = wb_df.astype(np.int64)
wb_df

,Afghanistan,Angola,Albania,Argentina,Armenia,Azerbaijan,Burundi,Benin,Burkina Faso,Bangladesh,Bulgaria,Bosnia and Herzegovina,Belarus,Belize,Bolivia,Brazil,Bhutan,Botswana,Central African Republic,China,Cote d'Ivoire,Cameroon,"Congo, Dem. Rep.","Congo, Rep.",Colombia,Comoros,Cabo Verde,Costa Rica,Djibouti,Dominica,Dominican Republic,Algeria,Ecuador,"Egypt, Arab Rep.",Eritrea,Ethiopia,Fiji,Gabon,Georgia,Ghana,Guinea,"Gambia, The",Guinea-Bissau,Grenada,Guatemala,Guyana,Honduras,Haiti,Indonesia,India,"Iran, Islamic Rep.",Jamaica,Jordan,Kazakhstan,Kenya,Kyrgyz Republic,Cambodia,Lao PDR,Lebanon,Liberia,St. Lucia,Sri Lanka,Lesotho,Morocco,Moldova,Madagascar,Maldives,Mexico,North Macedonia,Mali,Myanmar,Montenegro,Mongolia,Mozambique,Mauritania,Mauritius,Malawi,Niger,Nigeria,Nicaragua,Nepal,Pakistan,Panama,Peru,Philippines,Papua New Guinea,Paraguay,Romania,Russian Federation,Rwanda,Sudan,Senegal,Solomon Islands,Sierra Leone,El Salvador,Somalia,Serbia,Sao Tome and Principe,Eswatini,Syrian Arab Republic,Chad,Togo,Thailand,Tajikistan,Turkmenistan,Timor-Leste,Tonga,Tunisia,Turkey,Tanzania,Uganda,Ukraine,Uzbekistan,St. Vincent and the Grenadines,Vietnam,Vanuatu,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1960-01-01,979344507,7288778066,511485756,5893191433,133922025,35540000,14970400,43206645,20789505,146504255,772737948,2408825764,968283686,4084800,716253874,5984162468,325908,17358400,24142239,5798013609,373548111,145926448,345217589,118823126,2337070268,1172240,39376101,320105203,2594794,14433362,389548000,939559305,402122588,1801624657,29056800,169291507,11665440,103822382,79312483,572138164,336538268,5076000,0,16048000,192905000,82870994,158332049,52164000,3432691278,8425121119,4618942613,981947821,119092816,34977844,477537524,5527412,163589000,8345623,64232860,162259998,15516307,435678442,8122200,984947643,38504010,485146240,8884310,7097271963,1041777825,247329389,163680999,879825840,354767148,1318271500,26315139,31738400,135209089,31677128,836680592,242870748,3800810,3406742576,228889000,3299813044,2196027768,208775480,144131808,30147075,79671384011,4878977,385062510,145252948,11479143,61400031,205547537,73220144,2329427795,2183745,37033600,5388704719,35194310,39801356,1000572108,9731000,276435325,76035317,24548074,603200003,2746453200,197359339,151676389,550691959,59702042,1455000,25579000,3008900,2682000,75993698,48815506,21670999666,813521836,232362560
1961-01-01,979344507,7288778066,511485756,5893191433,133922025,35540000,14970400,43206645,20789505,146504255,772737948,2408825764,968283686,4084800,716253874,5984162468,325908,17358400,24142239,5798013609,373548111,145926448,345217589,118823126,2337070268,1172240,39376101,320105203,2594794,14433362,389548000,939559305,402122588,1801624657,29056800,169291507,11665440,103822382,79312483,572138164,336538268,5076000,0,16048000,192905000,82870994,158332049,52164000,3432691278,8425121119,4618942613,981947821,119092816,34977844,477537524,5527412,163589000,8345623,64232860,162259998,15516307,435678442,8122200,984947643,38504010,485146240,8884310,7097271963,1041777825,247329389,163680999,879825840,354767148,1318271500,26315139,31738400,135209089,31677128,836680592,242870748,3800810,3406742576,228889000,3299813044,2196027768,208775480,144131808,30147075,79671384011,4878977,385062510,145252948,11479143,61400031,205547537,73220144,2329427795,2183745,37033600,5388704719,35194310,39801356,1000572108,9731000,276435325,76035317,24548074,603200003,2746453200,197359339,151676389,550691959,59702042,1455000,25579000,3008900,2682000,75993698,48815506,21670999666,813521836,232362560
1962-01-01,979344507,7288778066,511485756,5893191433,133922025,35540000,14970400,43206645,20789505,146504255,772737948,2408825764,968283686,4084800,716253874,5984162468,325908,17358400,24142239,5798013609,373548111,145926448,345217589,118823126,2337070268,1172240,39376101,320105203,259479

### Functions

In [6]:
def get_ARMA_test_preds(country_name, train, test):
    y = train
    ARMAmodel = SARIMAX(y, order = (1, 1, 0))
    ARMAmodel = ARMAmodel.fit()
    
    y_pred = ARMAmodel.get_forecast(len(test.index))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = test.index
    y_pred_out = y_pred_df["Predictions"] 

    rmse = np.sqrt(mean_squared_error(test.values, y_pred_df["Predictions"]))
    #print(country_name, rmse)
    return rmse

In [7]:
def get_ARIMA_test_preds(country_name, train, test):
    y = train
    ARIMAmodel = ARIMA(y, order = (2, 2, 2))
    ARIMAmodel = ARIMAmodel.fit()
    
    y_pred = ARIMAmodel.get_forecast(len(test.index))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = test.index
    y_pred_out = y_pred_df["Predictions"] 

    rmse = np.sqrt(mean_squared_error(test.values, y_pred_df["Predictions"]))
    #print(country_name, rmse)
    return rmse

In [8]:
def get_SARIMA_test_preds(country_name, train, test):
    y = train
    SARIMAXmodel = SARIMAX(y, order = (1, 1, 0), seasonal_order=(2,2,2,6))
    SARIMAXmodel = SARIMAXmodel.fit()
    
    y_pred = SARIMAXmodel.get_forecast(len(test.index))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = test.index
    y_pred_out = y_pred_df["Predictions"] 

    rmse = np.sqrt(mean_squared_error(test.values, y_pred_df["Predictions"]))
    #print(country_name, rmse)
    return rmse

In [9]:
def get_ARMA_preds(country_name):
    y = wb_df[country_name]
    ARMAmodel = SARIMAX(y, order = (1, 1, 0))
    ARMAmodel = ARMAmodel.fit()
    y_pred = ARMAmodel.get_forecast(len(prediction_dates))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = prediction_dates
    y_pred_out = y_pred_df["Predictions"] 
    return y_pred_out

In [10]:
def get_ARIMA_preds(country_name):
    y = wb_df[country_name]
    ARIMAmodel = ARIMA(y, order = (2, 2, 2))
    ARIMAmodel = ARIMAmodel.fit()
    y_pred = ARIMAmodel.get_forecast(len(prediction_dates))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = prediction_dates
    y_pred_out = y_pred_df["Predictions"] 
    return y_pred_out

In [11]:
def get_SARIMA_preds(country_name):
    y = wb_df[country_name]
    SARIMAXmodel = SARIMAX(y, order = (1, 1, 0), seasonal_order=(2,2,2,6))
    SARIMAXmodel = SARIMAXmodel.fit()
    y_pred = SARIMAXmodel.get_forecast(len(prediction_dates))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = prediction_dates
    y_pred_out = y_pred_df["Predictions"] 
    return y_pred_out

In [12]:
# find index of min value in list
def find_min_index(number_list):
    max_value = min(number_list)
    return number_list.index(max_value)

### Run predictions

In [13]:
# used to train/test split
date_split = pd.to_datetime("2017-01-01", format='%Y-%m-%d')

In [14]:
# date time index for predictions: 2021, 2022, 2023
prediction_dates = pd.date_range('2021-01-01', periods=3, freq='AS')
prediction_dates

DatetimeIndex(['2021-01-01', '2022-01-01', '2023-01-01'], dtype='datetime64[ns]', freq='AS-JAN')

In [15]:
import warnings
# supress warnings while processing files
warnings.filterwarnings('ignore')

def get_predictions():
    total_preds = {}
    # for each country
    #for country_name in ["Chad"]:
    for country_name in tqdm(wb_df.columns.unique()):

        # split series
        train = wb_df[country_name][wb_df.index <= date_split]
        test = wb_df[country_name][wb_df.index > date_split]

        # get model scores
        arma_score = get_ARMA_test_preds(country_name, train, test)
        arima_score = get_ARIMA_test_preds(country_name, train, test)
        sarima_score = get_SARIMA_test_preds(country_name, train, test)

        # select the model with lowest error
        best_score = find_min_index([arma_score, arima_score, sarima_score])

        # use best model to predict 2021, 2022, 2023
        if(best_score == 0):
            best_preds = get_ARMA_preds(country_name)
        elif(best_score == 1):
            best_preds = get_ARIMA_preds(country_name)
        elif(best_score == 2):
            best_preds = get_SARIMA_preds(country_name)
    
        # to plot the values nicely we add the latest row (2020) from the original dataset to the predictions
        latest_obs = wb_df.index.max()
        best_preds.loc[latest_obs] = wb_df[country_name].loc[latest_obs]
        # enforce sort by date
        best_preds = best_preds.sort_index()
        
        # save predictions to dict
        best_preds = best_preds.astype(np.int64)
        total_preds[country_name] = best_preds

        fig, ax = plt.subplots()
        # plot only last 30 values
        plt.plot(wb_df[country_name][-15:], color = "black", label = 'Observed')
        plt.plot(best_preds, color='Blue', label = 'Forecast')

        # format plot
        func = lambda y, pos: f"${y:,.0f}"
        ax.yaxis.set_major_formatter(func)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.xaxis.set_major_locator(mdates.YearLocator(1))
        plt.ylabel('Debt in current USD')
        plt.xlabel('Year')
        plt.xticks(rotation=45)
        plt.title("Debt forecast: "+ country_name)
        plt.grid()
        plt.legend()
        
        # save plot to file
        plt.savefig("../debt-plots/"+ country_name +'.png',dpi=300, transparent=True, bbox_inches='tight')
        plt.close()

    return pd.DataFrame.from_dict(total_preds)

predictions = get_predictions()
predictions

100%|██████████| 122/122 [01:16<00:00,  1.59it/s]


,Afghanistan,Angola,Albania,Argentina,Armenia,Azerbaijan,Burundi,Benin,Burkina Faso,Bangladesh,Bulgaria,Bosnia and Herzegovina,Belarus,Belize,Bolivia,Brazil,Bhutan,Botswana,Central African Republic,China,Cote d'Ivoire,Cameroon,"Congo, Dem. Rep.","Congo, Rep.",Colombia,Comoros,Cabo Verde,Costa Rica,Djibouti,Dominica,Dominican Republic,Algeria,Ecuador,"Egypt, Arab Rep.",Eritrea,Ethiopia,Fiji,Gabon,Georgia,Ghana,Guinea,"Gambia, The",Guinea-Bissau,Grenada,Guatemala,Guyana,Honduras,Haiti,Indonesia,India,"Iran, Islamic Rep.",Jamaica,Jordan,Kazakhstan,Kenya,Kyrgyz Republic,Cambodia,Lao PDR,Lebanon,Liberia,St. Lucia,Sri Lanka,Lesotho,Morocco,Moldova,Madagascar,Maldives,Mexico,North Macedonia,Mali,Myanmar,Montenegro,Mongolia,Mozambique,Mauritania,Mauritius,Malawi,Niger,Nigeria,Nicaragua,Nepal,Pakistan,Panama,Peru,Philippines,Papua New Guinea,Paraguay,Romania,Russian Federation,Rwanda,Sudan,Senegal,Solomon Islands,Sierra Leone,El Salvador,Somalia,Serbia,Sao Tome and Principe,Eswatini,Syrian Arab Republic,Chad,Togo,Thailand,Tajikistan,Turkmenistan,Timor-Leste,Tonga,Tunisia,Turkey,Tanzania,Uganda,Ukraine,Uzbekistan,St. Vincent and the Grenadines,Vietnam,Vanuatu,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
2020-01-01,3036369312,67286794415,10859239096,253760056780,13093864395,15811923784,625985283,5250479649,4494781776,67749043920,43347397245,14263489883,42491219492,1526004210,15421072053,549234311520,2868979841,1597427934,935630034,2349388621678,25072693104,13863899165,6137246433,5253134974,155171767040,301131309,2069786254,31287627814,2678724034,328975990,44467742286,5178215526,56190547675,131579729752,788537429,30364411863,1518014488,7615875628,20088956454,31323084616,4175376873,775654200,805942506,658742963,25136104794,1506687262,11016615220,2317684493,417531650069,564179014212,5451188395,18034588389,38016104454,162974379888,38193646113,8697096132,17562061636,17164137036,68865417339,1480598762,733837563,56341952149,1052218388,65682954942,8475902591,4873178319,3351712547,467511920765,10606666542,6081077523,13348140041,9722712647,33236211399,20932324154,5710104834,18527475048,2943281966,4590696470,70570530052,12050448081,7904918393,116505539223,108866240274,73549205266,98479781366,17970999839,19763367609,142389651029,475518171675,8193366972,22953534423,17238618348,428927088,2113502029,18333708634,4659652464,38467088092,291188749,766084561,4763248268,3654297383,2546422966,204146618941,6797818017,5636041535,231800534,194304203,41038099975,435889447921,25537812220,17206774508,129899079724,32174405287,406295764,125045334156,455667428,437296399,3108497520,7120836024,170766641061,30045885685,12740664732
2021-01-01,3172930266,69530776214,11476099065,241444180159,15284206734,15803531552,629565718,5957167841,4774252168,78004340328,45211546620,14738719070,43504080869,1549715569,15967945909,537613406194,3103212020,1603324352,952526223,2160293737009,28899189480,14446429014,6159980847,5249887121,165163863867,303942481,2306914507,33196590836,2864549822,353190079,49044613038,5034495139,60318799545,147768832454,789727358,31529949195,1658266377,7905471975,20992880752,35400017612,4221738070,772537963,924895337,647357451,25198130557,1505998126,11811581361,2359750114,429732809167,593924926978,5615117156,18419779722,40475939875,164941609494,42938769335,8832224495,19786107278,17537980446,66916982783,1602665337,703197563,56506189854,1081454623,70617670983,8597105057,4987376507,3812596958,464658228320,11160616654,6589083181,14452083655,10576255946,34630367048,21577958950,5759823451,15650270032,3056980405,4964139281,79177306829,12214513591,9210940200,123518263695,115408200710,74860076380,101125063323,17628991472,24771324809,158388028222,473457544097,9494905696,23238374322,19194125375,460163234,2206702867,18877131961,4377093798,41402735079,295483693,719869444,4763740756,3625026666,2956367339,219463764566,7135531205,5102048128,258569605,211657137,41935506258,433446335323,26933813210,19008494818,132851293647,41896001821,420856843,134860500089,480560827,4

## Save predictions

In [16]:
predictions.index.name = "date"

In [17]:
predictions.to_csv("debt_predictions.csv")

In [ ]:
# save json
predictions_json = predictions.to_json(orient= "index")
with open('debt_predictions.json', 'w') as outfile:
    json.dump(predictions_json, outfile)